In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

import sys

sys.path.append("/gdrive/My Drive")
sys.path.append('..')
sys.path.append('../..')

In [2]:
import time

import copy
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import csv
import glob

import pandas as pd
from keras import Input, Model, metrics
from keras.layers import LocallyConnected2D, Concatenate, Conv2D, ZeroPadding2D
from shapely.geometry import Point, Polygon

use_optimal_permutation = True
WIND_path = "/gdrive/My Drive/data.pkl"

nongrid_path = "/gdrive/My Drive/MS_winds_new.dat"
nongrid_permutations_path = "/gdrive/My Drive/top10perms_GA.pkl"
save_results_path = "/gdrive/My Drive"

copernicus_path = "/gdrive/My Drive/formed_cop_shifted_data.pkl"

Using TensorFlow backend.


In [0]:
def distsqr(a, b):
    return (a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2

def prepare_sets(data, look_back, horizon, nr_predictions, train_cnt, valid_cnt, test_cnt):
    sets, sets_sizes = [], [train_cnt, valid_cnt, test_cnt]
    curr_s = 0

    for i in range(len(sets_sizes)):
        x_out = None
        for j in range(look_back):
            x_temp = np.expand_dims(data[curr_s + j:curr_s + j + sets_sizes[i]], 3)
            x_out = x_temp if x_out is None else np.concatenate((x_out, x_temp), axis=3)
        y_out = None
        for j in range(nr_predictions):
            y_out_temp = np.expand_dims(data[curr_s + j + look_back + horizon:curr_s + j + sets_sizes[i] + look_back + horizon], 3)
            y_out = y_out_temp if y_out is None else np.concatenate((y_out, y_out_temp), axis=3)

        sets.append((x_out, y_out))
        curr_s += sets_sizes[i] + look_back + horizon
    return sets

def get_non_grid_winds_dataset(data, look_back, horizon=0, normalize=True):
    min_val, max_val = np.min(data), np.max(data)

    reshaped = []
    for i in range(0, data.shape[0]):
        curr = np.zeros((8, 8))
        iter = 0
        for j in range(8):
            for k in range(8):
                if iter >= 57:
                    break
                curr[j, k] = data[i, iter]
                iter += 1
        reshaped.append(curr)
    reshaped = np.stack(reshaped, axis=0)

    if normalize:
        reshaped = (reshaped - min_val) / (max_val - min_val)

    train_cnt, valid_cnt, test_cnt = 5664, 300, 361

    sets = prepare_sets(reshaped, look_back, horizon, 6, train_cnt, valid_cnt, test_cnt)

    return sets[0][0], sets[0][1], sets[1][0], sets[1][1], sets[2][0], sets[2][1], min_val, max_val, reshaped


def split_sets(dicts, look_back=5, horizon = 1, normalize=True):
    dim = int(np.sqrt(len(dicts)))

    mapping = {}
    index = 0

    for i in range(dim):
        high_el, melo = -1e9, 0
        for key, val in dicts.items():
            if key not in mapping and key[1] > high_el:
                melo = key
                high_el = key[1]
        mapping[melo] = index
        index += 1
        for j in range(dim - 1):
            k, disto = 0, 1e9
            for key, val in dicts.items():
                if key not in mapping and key[0] < melo[0] and distsqr(key, melo) < disto:
                    disto = distsqr(key, melo)
                    k = key
            melo = k
            mapping[k] = index
            index += 1

    for key, val in mapping.items():
        mapping[key] = ((val % dim) * dim + int(val // dim)) % (dim * dim)

    plt.figure()
    for key, val in mapping.items():
        y, x = int(val / dim), val % dim,
        plt.text(key[0], key[1], str((y, x)), fontsize=12)
        plt.plot(key[0], key[1], ".")
    plt.show()

    data = np.zeros((25920, dim, dim))

    for key, val in dicts.items():
        id = mapping[key]
        data[:, int(id / dim), id % dim] = copy.deepcopy(val["Wind"].values[:])

    min_val, max_val = data.min(), data.max()
    if normalize:
        data = (data - min_val) / (max_val - min_val)

    train_cnt, valid_cnt, test_cnt = 15552 - look_back - horizon, 5184 - look_back - horizon, 5184 - horizon - look_back
    sets = prepare_sets(data, look_back, horizon, 1, train_cnt, valid_cnt, test_cnt)

    return sets[0][0], sets[0][1], sets[1][0], sets[1][1], sets[2][0], sets[2][1], min_val, max_val, data


def split_sets_copernicus(data, normalize=True, horizon=0, split_prop=[0.7, 0.1, 0.2], look_back=12):
    min_val, max_val = data.min(), data.max()
    if normalize:
        data = (data - min_val) / (max_val - min_val)

    train_cnt, valid_cnt, test_cnt = int(data.shape[0] * split_prop[0]), int(data.shape[0] * split_prop[1]), int(data.shape[0] * split_prop[2])

    sets = prepare_sets(data, look_back, horizon, 1, train_cnt - look_back - horizon, valid_cnt - look_back - horizon, test_cnt - look_back)
    for ii in sets:
      for jj in ii:
        print(len(jj))
    return sets[0][0], sets[0][1], sets[1][0], sets[1][1], sets[2][0], sets[2][1], min_val, max_val, data

In [0]:
def get_errors(predicted, test_y,  min_val, max_val):
    total_nodes = predicted.shape[1] * predicted.shape[2]

    errs = np.zeros((predicted.shape[1], predicted.shape[2], predicted.shape[3]))
    mape = np.zeros((predicted.shape[1], predicted.shape[2], predicted.shape[3]))
    mae = np.zeros((predicted.shape[1], predicted.shape[2], predicted.shape[3]))

    for i in range(test_y.shape[0]):
        denorm_truth = test_y[i, :, :, :] * (max_val - min_val) + min_val
        denorm_pred = predicted[i, :, :, :] * (max_val - min_val) + min_val

        mape += np.abs((denorm_truth - denorm_pred) / denorm_truth)
        errs += (denorm_truth - denorm_pred) ** 2
        mae += np.abs(denorm_pred - denorm_truth)
    errs = np.sum(errs, axis=2) / test_y.shape[0]

    rmse = np.sqrt(np.sum(errs) / total_nodes)
    mape = np.sum(np.sum(mape, axis=2) / test_y.shape[0]) / total_nodes * 100
    mae = np.sum((np.sum(mae, axis=2) / test_y.shape[0])) / total_nodes

    return rmse, mape, mae

def get_errors_non_grid(predicted, test_y, min_val, max_val):
    nr_predictions = predicted.shape[-1]

    errs = np.zeros((8, 8, nr_predictions))
    mape = np.zeros((8, 8, nr_predictions))
    mae = np.zeros((8, 8, nr_predictions))

    test_iter = 0
    for i in range(test_y.shape[0]):
        if i % nr_predictions != 0:
            continue
        test_iter += nr_predictions
        denorm_truth = test_y[i, :, :, :] * (max_val - min_val) + min_val
        denorm_pred = predicted[i, :, :, :] * (max_val - min_val) + min_val

        mape += np.abs((denorm_truth - denorm_pred) / denorm_truth)
        errs += (denorm_truth - denorm_pred) ** 2
        mae += np.abs(denorm_pred - denorm_truth)
    errs = np.sum(errs, axis=2)
    errs /= test_iter
    rmse = np.sqrt(np.sum(errs) / 57)
    mape = np.sum(mape, axis=2)
    mape /= test_iter
    mape = np.sum(mape) / 57
    mape *= 100

    mae = np.sum(mae, axis=2)
    mae /= test_iter

    mae = np.sum(mae) / 57

    return rmse, mape, mae
########################################################################################
def read_non_grid_data():
    file_dataset = nongrid_path
    with open(file_dataset) as f:
        data = csv.reader(f, delimiter=",")
        winds = []
        for line in data:
            winds.append((line))
    winds = (np.array(winds)).astype(float)  # all
    return winds

def read_folder(folder_path):
    os.chdir(folder_path)

    results = {}
    names = {}
    for file in reversed(sorted(glob.glob("*"))):
        if not file.endswith(".csv"):
            continue
        dataframe = pd.read_csv(file, skiprows=4, header=None)
        dataframe.columns = ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Wind']
        # read long and lat
        iters = 0
        with open(file, 'r') as content_file:
            content = content_file.read().split("\n")

            long, lat = float(content[1].split(',')[1]), float(content[2].split(',')[1])
        names[(long, lat)] = file
        results[(long, lat)] = dataframe
    return results, names

def read_gridded_pickle(debug=True, dataset_id=0, for_testsuit=False):
    dict, names = pickle.load(open(WIND_path, "rb"))

    # take only whats needed
    count = 0
    polygon = Polygon([(-85.1923, 40.4192), (-84.9561, 40.396), (-84.9909, 40.2155), (-85.2278, 40.2404)])

    rev_names = {}
    for key, val in names.items():
        rev_names[val] = 0
    names_in = {}

    removal = []

    for key, val in dict.items():
        pnt = Point(key[0], key[1])

        if polygon.contains(pnt):
            names_in[key] = 1
            plt.plot(key[0], key[1], '.', markersize=8, color="red")
            count += 1
        else:
            removal.append(key)
            plt.plot(key[0], key[1], '.', markersize=2, color="grey")
    assert count == 100, "Error: There should be 100 matched points!"
    plt.show()

    mino, maxo = 1e9, -1e9
    for key, val in dict.items():
        if val is None:
            continue
        dict[key] = val[(val.Month <= 3)]

        mino = min(mino, dict[key]["Wind"].min())
        maxo = max(maxo, dict[key]["Wind"].max())

    assert mino == 0.048 and maxo == 27.228, "Error: min/max values are not as reported"
    return dict

In [0]:
########################################################################################################################
class CNN():
    def __init__(self, nr_predictions, train_x, train_y, valid_x, valid_y, test_x, test_y, layers=[(10,10)]):
        self.input_dim, self.look_back, self.nr_predictions = (train_x[0].shape[1], train_x[0].shape[2]), train_x[0].shape[-1], nr_predictions
        self.train_x, self.train_y, self.valid_x, self.valid_y, self.test_x, self.test_y = train_x, train_y, valid_x, valid_y, test_x, test_y
        self.layers = layers

    def get_model(self):
        I1 = Input(shape=(self.input_dim[0], self.input_dim[1], self.look_back))

        for idx, layer in enumerate(self.layers):
            if idx == 0:
                Conved = Conv2D(layer[1], kernel_size=layer[0], activation='relu', padding="same")(I1)
            else:
                Conved = Conv2D(layer[1], kernel_size=layer[0], activation='relu', padding="same")(Conved)

        Sing = Conv2D(self.nr_predictions, kernel_size=1, padding="same")(Conved)
        model = Model(inputs=[I1], outputs=[Sing])
        model.compile(optimizer='adam', loss='mean_squared_error', metrics=[metrics.mse])

        self.model = model
        return self.model

    def train(self, epochs=1):
        print(self.train_x[0].shape)
        print(self.train_y.shape)

        hist = self.model.fit(self.train_x, self.train_y, validation_data=(self.valid_x, self.valid_y), epochs=epochs, batch_size=200, verbose=2)
        return hist

    def predict(self, data):
        predicted = self.model.predict(data)
        return predicted
# ======================================================================================================================
class LILWCNN():
    def __init__(self, nr_predictions, train_x, train_y, valid_x, valid_y, test_x, test_y, learnable_inputs_count, local_weights_count, local_weight_receptive_field=(1,1),
                 be_persistent=False, use_input=True, layers=(10,10)):
        self.nr_predictions = nr_predictions
        self.train_x, self.train_y, self.valid_x, self.valid_y, self.test_x, self.test_y = train_x, train_y, valid_x, valid_y, test_x, test_y

        self.input_dim = (train_x[0].shape[1], train_x[0].shape[2])
        self.look_back = train_x[0].shape[3]

        self.learnable_inputs_count= learnable_inputs_count
        self.local_weights_count= local_weights_count

        self.local_weight_receptive_field = local_weight_receptive_field
        self.be_persistent=be_persistent
        self.use_input=use_input
        self.layers = layers
        assert learnable_inputs_count > 0 or local_weights_count > 0, "Error: not using learnable inputs and/or local weights, use vanilla CNN instead"

    def get_model(self):
        I1 = Input(shape=(self.input_dim[0], self.input_dim[1], self.look_back))
        I2 = Input(shape=(self.input_dim[0], self.input_dim[1], 1))

        if(self.local_weights_count > 0 and self.local_weight_receptive_field == (2,2)):
            # We 0-pad the input, since locally connected only supports `valid` padding.
            # Note that this only works for input shapes described in the paper
            Padding = ZeroPadding2D(padding=((0, 1), (0, 1)))(I1)
        else:
            Padding = I1

        K = None

        if self.local_weights_count > 0:
            C1 = LocallyConnected2D(self.local_weights_count, kernel_size=self.local_weight_receptive_field, name="local_weight",
                                input_shape=(self.input_dim[0], self.input_dim[1], self.look_back), use_bias=False)(Padding)
            K = C1
        if self.learnable_inputs_count > 0:
            C2 = LocallyConnected2D(self.learnable_inputs_count, kernel_size=(1, 1), name="learnable_input",
                                input_shape=(self.input_dim[0], self.input_dim[1], 1), use_bias=False)(I2)
            K = C2 if K is None else Concatenate(-1)([K, C2])

        Merged = Concatenate(-1)([I1, K]) if self.use_input else K

        Conved = Conv2D(self.layers[0][1], kernel_size=self.layers[0][0], activation='relu', padding="same")(Merged)

        for idx, layer in enumerate(self.layers[1:]):
            Concat = Concatenate(-1)([Conved, K]) if self.be_persistent else Conved

            Conved = Conv2D(layer[1], kernel_size=layer[0], activation='relu', padding="same")(Concat)

        Sing = Conv2D(self.nr_predictions, kernel_size=1, padding="same")(Conved)
        model = Model(inputs=[I1, I2], outputs=[Sing])
        model.compile(optimizer='adam', loss='mean_squared_error', metrics=[metrics.mse])

        self.model = model
        return self.model

    def train(self, epochs=1):
        hist = self.model.fit(self.train_x, self.train_y, validation_data=(self.valid_x, self.valid_y), epochs=epochs, batch_size=200, verbose=2)
        return hist

    def predict(self, data):
        predicted = self.model.predict(data)
        return predicted

# ======================================================================================================================

In [0]:
  def get_coords(in_arr):
      for i in range(in_arr.shape[1]):
          for j in range(in_arr.shape[2]):
              in_arr[:, i, j, 0] = i / float(in_arr.shape[1] - 1)
              in_arr[:, i, j, 1] = j / float(in_arr.shape[2] - 1)
      return in_arr

# Experiment driver

In [0]:
# ======================================================================================================================
def run_experiment(task_name, predict_horizon, nr_predictions, look_back, use_optimal_permutation=False):
    for this_horizon in predict_horizon:
        if task_name == "WIND":
            data = read_gridded_pickle(False)

            train_x, train_y, valid_x, valid_y, test_x, test_y, min_val, max_val, _ = split_sets(data,
                                                                                                 look_back=look_back,
                                                                                                 horizon=this_horizon)
        elif task_name == "nongrid":
            data = read_non_grid_data()

            train_x, train_y, valid_x, valid_y, test_x, test_y, min_val, max_val, raw_data = get_non_grid_winds_dataset(
                data, look_back=look_back, horizon=this_horizon, normalize=True)

            if use_optimal_permutation:
                # ---------- Permute by mutual information

                # load best perms
                perms = pickle.load(open(nongrid_permutations_path, "rb"))[0]

                train_x_perm, train_y_perm = np.zeros_like(train_x), np.zeros_like(train_y)
                valid_x_perm, valid_y_perm = np.zeros_like(valid_x), np.zeros_like(valid_y)
                test_x_perm, test_y_perm = np.zeros_like(test_x), np.zeros_like(test_y)

                for i in range(8):
                    for j in range(8):
                        id = i * 8 + j
                        if perms[id] >= 57:
                            continue
                        perms_y, perms_x = perms[id] // 8, perms[id] % 8

                        train_x_perm[:, i, j, :] = train_x[:, perms_y, perms_x, :]
                        train_y_perm[:, i, j, :] = train_y[:, perms_y, perms_x, :]

                        valid_x_perm[:, i, j, :] = valid_x[:, perms_y, perms_x, :]
                        valid_y_perm[:, i, j, :] = valid_y[:, perms_y, perms_x, :]

                        test_x_perm[:, i, j, :] = test_x[:, perms_y, perms_x, :]
                        test_y_perm[:, i, j, :] = test_y[:, perms_y, perms_x, :]

                train_x, train_y = np.copy(train_x_perm), np.copy(train_y_perm)
                valid_x, valid_y = np.copy(valid_x_perm), np.copy(valid_y_perm)
                test_x, test_y = np.copy(test_x_perm), np.copy(test_y_perm)
                # --------------------------------------------------------------------
        elif task_name == "copernicus":
            whole_data = pickle.load(open(copernicus_path, "rb"))

            train_x, train_y, valid_x, valid_y, test_x, test_y, min_val, max_val, _ = split_sets_copernicus(
                whole_data["data"][:, :, :, 0],
                look_back=look_back,
                horizon=this_horizon)

        input_dim = (train_x.shape[1], train_x.shape[2]) # spatial dimension

        train_ones = np.ones((train_x.shape[0], input_dim[0], input_dim[1], 1))
        valid_ones = np.ones((valid_x.shape[0], input_dim[0], input_dim[1], 1))
        test_ones = np.ones((test_x.shape[0], input_dim[0], input_dim[1], 1))


        train_coords, valid_coords, test_coords = np.ones((train_x.shape[0], input_dim[0], input_dim[1], 2)), np.ones((valid_x.shape[0], input_dim[0], input_dim[1], 2)), np.ones((test_x.shape[0], input_dim[0], input_dim[1], 2))
        train_coords, valid_coords, test_coords = get_coords(train_coords), get_coords(valid_coords), get_coords(test_coords)

        print(train_y.shape)
        models = [
            {
                "name": "Persistent LI+LW CNN",
                "skip": True,
                "model": LILWCNN(nr_predictions, [train_x, train_ones], train_y, [valid_x, valid_ones], valid_y, [test_x, test_ones],
                               test_y, 2, 2, be_persistent=True, layers=[(5, 22), (4, 22), (3, 22)])
            },
            {
                "name": "LI CNN",
                "skip": True,
                # 2 learnable inputs are used
                "model": LILWCNN(nr_predictions, [train_x, train_ones], train_y, [valid_x, valid_ones], valid_y, [test_x, test_ones],
                               test_y, 2, 0, layers=[(5, 28), (4, 30), (3, 30)])
            },
            {
                "name": "CoordConv",
                "skip": True,
                # coordinates are concatinated with the input
                "model": CNN(nr_predictions, [np.concatenate((train_x, train_coords), axis=-1)], train_y,
                             [np.concatenate((valid_x, valid_coords), axis=-1)], valid_y,
                             [np.concatenate((test_x, test_coords), axis=-1)], test_y, layers=[(5, 28), (4, 30), (3, 30)])
            },
            {
                "name": "LI+LW CNN",
                "skip": True,
                # 2 learnable inputs and local weights are used
                "model": LILWCNN(nr_predictions, [train_x, train_ones], train_y, [valid_x, valid_ones], valid_y, [test_x, test_ones],
                               test_y, 2, 2, layers=[(5, 28), (4, 30), (3, 30)])
            },
        ]

        epochs_count = 100
        extra_text = ""

        repeat_exp_count = 1

        iter = 0
        for exp_iter in range(repeat_exp_count):
            print("Starting {} / {} experiments batch".format(exp_iter, repeat_exp_count))
            for model in models:

                if task_name == "nongrid" and use_optimal_permutation and "Permuted " not in model["name"]:
                    model["name"] = "Permuted {}".format(model["name"])

                if "skip" in model and model["skip"] is True:
                    print("Skipping {} ".format(model["name"]))
                    continue
                # initialize
                model["model"].get_model()
                model["model"].model.summary()
                print("Training {} model, with {} learnable parameters".format(model["name"], model["model"].model.count_params()))

                train_error, valid_error, test_error = [], [], []
                # train for epochs or something
                for epoch in range(epochs_count):
                    hist = model["model"].train()
                    print("Trained")

                    predicted = model["model"].predict(model["model"].test_x)
                    if task_name == "WIND":
                        test_rmse, test_mape, test_mae = get_errors(predicted, model["model"].test_y, min_val, max_val)
                    elif task_name == "nongrid":
                        if use_optimal_permutation and "Permuted" in model["name"]:  # hacky! this holds only for the permutation used in the paper
                            predicted[:, :-2, -1, :] = 0
                            predicted[:, 0, -2:, :] = 0
                        else:
                            predicted[:, 7, 1:, :] = 0
                        #
                        test_rmse, test_mape, test_mae = get_errors_non_grid(predicted, model["model"].test_y, min_val, max_val)
                    elif task_name == "copernicus":
                        test_rmse, test_mape, test_mae = get_errors(predicted, model["model"].test_y, min_val, max_val)

                    print(" Model - {} , epoch {} / {}.".format(model["name"], epoch + 1, epochs_count))

                    print("Test RMSE - {}, MAPE - {}, MAE - {}".format(test_rmse, test_mape, test_mae))
                    test_error.append((test_rmse, test_mape, test_mae))
                    train_error.append((hist.history["loss"]))
                    valid_error.append((hist.history["val_loss"]))

                folder = "{}/results/real_world/{}/".format(save_results_path, task_name)

                if not os.path.isdir(folder):
                    os.makedirs(folder)

                file_name = "{}/{}_{}_{}_{}_{}_{}".format(folder, task_name,
                                                                  model["name"],
                                                                  model["model"].model.count_params(),
                                                                  epochs_count,
                                                                  str(time.ctime()).replace(" ", "_").replace(":", "-"),
                                                                  this_horizon)

                pickle.dump({"name": model["name"],
                             "learnable_parameters": model["model"].model.count_params(),
                             "nr_predictions": nr_predictions,
                             "predict_horizon": this_horizon,
                             "look_back": look_back,
                             "summary": model["model"].model.to_json(),
                             "model_descriptor": model["model"].model.to_json(),
                             "test_error": test_error,
                             "time": time.ctime(),
                             "valid_error": valid_error,
                             "train_error": train_error},
                            open(file_name, "wb"))


In [0]:
  # Use id = "WIND" for the first experiment in the paper
  #        = "nongrid" for the second experiment in the paper
  #        = "copernicus" for the third experiment in the paper

# run_experiment("nongrid", [0], 6, 12, use_optimal_permutation=False)
# run_experiment("nongrid", [0], 6, 12, use_optimal_permutation=True)
# run_experiment("copernicus", [0, 1], 1, 8)
run_experiment("WIND", [0, 1, 2, 3, 5, 11], 1, 8)


#Results

In [0]:
def get_results(task_name):
  possible_horizons = {}
  def read_folder(folder_path):
      os.chdir(folder_path)

      results = {}
      for file in reversed(sorted(glob.glob("*"))):
          if os.path.isdir(file):
              continue
          res = pickle.load(open(file, "rb"))
          key = (res["name"], res["learnable_parameters"], len(res["train_error"]))

          if key not in results:
              results[key] = {}
          if res["predict_horizon"] not in results[key]:
              results[key][res["predict_horizon"]] = {}
              results[key][res["predict_horizon"]]["runs"] = []

          results[key][res["predict_horizon"]]["runs"].append(res)
      return results

  folder = "{}/results/real_world/{}/".format(save_results_path, task_name)

  results = read_folder(folder)
  for key, vals in results.items():
      for horizon_key, val in vals.items():
          valid_errs, test_mse_errs, learnable_params = None, None, []

          for run in val["runs"]:
              use_epochs = len(run["test_error"])

              possible_horizons[run["predict_horizon"]] = 1

              learnable_params.append(run["learnable_parameters"])
              valid_errs = np.array(run["valid_error"][:use_epochs]) if valid_errs is None else np.hstack((valid_errs,np.array(run["valid_error"][:use_epochs])))
              test_errs = np.array(run["test_error"][:use_epochs])[:,0][None].T if test_mse_errs is None else np.hstack((test_mse_errs,np.array(run["test_error"][:use_epochs])[:,0][None].T))

          learnable_params = np.mean(learnable_params).astype(int)
          def get_batch_stats(batch):
              mean_batch = np.mean(batch, axis=-1)

              min_batch = np.min(batch, axis=-1)
              max_batch = np.max(batch,axis=-1)
              std_batch = np.std(batch, axis=-1)

              return std_batch, mean_batch, min_batch, max_batch

          std_test, mean_test, min_test, max_test = get_batch_stats(test_errs)
          std_valid, mean_valid, min_valid, max_valid = get_batch_stats(valid_errs)

          best_valid_id = np.argmin(mean_valid)

          valid_errs = np.array(valid_errs)
          if len(valid_errs.shape) == 1:
              valid_errs = np.expand_dims(valid_errs, axis=-1)
          best_valid_individ_ids = np.atleast_2d(np.argmin(valid_errs, axis=0))

          indexes = []

          cor_valids, cor_tests = [], []

          for col, index in enumerate(best_valid_individ_ids.flatten()):
              cor_tests.append(test_errs[index, col])
              cor_valids.append(valid_errs[index,col])

          cor_tests = np.array(cor_tests)
          best_test_individ_mean = np.mean(cor_tests)
          best_test_individ_std = np.std(cor_tests)

          best_valid_mean = np.mean(cor_valids)
          results[key][horizon_key]["formed_sting"] = " | {:<15} | {:<15}  ".format(round(best_valid_mean,4), round(best_test_individ_mean, 4))
  # form table
  names = sorted(list(results.keys()), key=lambda x: x[0].replace("Permuted ", ""))
  for key in names:
      vals = results[key]
      horizons_list = sorted(list(possible_horizons.keys()))
      lines = [ "{:>40} {:<10}".format(key[0], str(key[1]) )]

      for horizon in horizons_list:
          if horizon not in vals:
              lines.append("- & -")
              continue
          lines.append(vals[horizon]["formed_sting"])
      
      total_len = 0
      for index, j in enumerate(lines):
          spc = "  "
          if index == 0:
              spc = "   "
          print(spc + j , end="",flush=True)
          total_len += len(j)
      print()
      print("-"*total_len)



In [0]:
# get_results("copernicus")
# get_results("WIND")
# get_results("nongrid")